## making the api

In [ ]:
import yfinance as yf
import pandas as pd

# List of 10 stock tickers (companies)
tickers = ['MSFT', 'AAPL', 'GOOG', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'INTC', 'BABA']

# Empty list to store all the stock data
all_stock_data = []

# Download stock data for each company and append to the list
for ticker in tickers:
    stock_data = yf.download(ticker, start="2010-01-01", end="2023-01-01")

    # Add a new column to the data with the ticker name
    stock_data['Ticker'] = ticker

    # Append the data to the list
    all_stock_data.append(stock_data)

# Concatenate all the data into one DataFrame
combined_stock_data = pd.concat(all_stock_data)

# Save the combined data to a CSV file
combined_stock_data.to_csv('combined_stock_data.csv')

print("Stock data saved to 'combined_stock_data.csv'")


////

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Load your data
df = pd.read_csv('dataset.csv', parse_dates=['Date'])

# Sort by date (in case it's not sorted)
df = df.sort_values(by=['Ticker', 'Date'])

# Feature Engineering - Adding moving averages as an example
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_20'] = df['Close'].rolling(window=20).mean()

# Drop rows with NaN values created by rolling
df = df.dropna()

# Use 'Close' as target variable and other features
features = ['Open', 'High', 'Low', 'Volume']
target = 'Close'

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features])

# Prepare the dataset for time-series forecasting
def create_dataset(data, target_col, look_back=60):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back, target_col])
    return np.array(X), np.array(y)

# Create the time-series dataset
X, y = create_dataset(scaled_data, df.columns.get_loc(target))

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict and evaluate
y_pred = model.predict(X_test)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0125
Epoch 2/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.0077
Epoch 3/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0065
Epoch 4/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0060
Epoch 5/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0055
Epoch 6/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0051
Epoch 7/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0049
Epoch 8/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0047
Epoch 9/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.0045
Epoch 10/10
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0045
193/193 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Mean Absolute Error: 138197576.0064876


In [2]:
# Correctly invert scaling for the predictions and true values
y_test_rescaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]
y_pred_rescaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_pred), axis=1))[:, -1]

mae = np.mean(np.abs(y_test_rescaled - y_pred_rescaled))
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 138197576.0064876


using LSTM

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('dataset.csv', parse_dates=['Date'])

# Sort by date (in case it's not sorted)
df = df.sort_values(by=['Ticker', 'Date'])

# Feature Engineering - Adding moving averages as an example
df['MA_5'] = df['Close'].rolling(window=5).mean()
df['MA_20'] = df['Close'].rolling(window=20).mean()

# Drop rows with NaN values created by rolling
df = df.dropna()

# Use 'Close' as the target variable and other features
features = ['Open', 'High', 'Low', 'Volume', 'MA_5', 'MA_20']
target = 'Close'

# Normalize the data (scaling the features)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features])

# Prepare the dataset for time-series forecasting
def create_dataset(data, target_col, look_back=60):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back])
        y.append(data[i + look_back, target_col])
    return np.array(X), np.array(y)

# Create the time-series dataset (X and y)
X, y = create_dataset(scaled_data, df.columns.get_loc(target))

# Train-Test Split (chronologically)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Predict the test set
y_pred = model.predict(X_test)

# Invert scaling for predictions and true values
# To invert the scaling, concatenate the features (X_test) with the predictions and true values
y_test_rescaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]
y_pred_rescaled = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_pred), axis=1))[:, -1]

# Evaluation
mae = np.mean(np.abs(y_test_rescaled - y_pred_rescaled))
rmse = np.sqrt(np.mean((y_test_rescaled - y_pred_rescaled)**2))

print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


769/769 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.0115
Epoch 2/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.0068
Epoch 3/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0063
Epoch 4/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0059
Epoch 5/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.0053
Epoch 6/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0049
Epoch 7/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 0.0045
Epoch 8/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.0040
Epoch 9/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0040
Epoch 10/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0038
Epoch 11/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0036
Epoch 12/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 0.0033
Epoch 13/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 0.0034
Epoch 14/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0035
Epoch 15/50
769/769 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0

In [6]:
from google.colab import files
# Save the model
model.save('stock_prediction_model.h5')

# Download the saved model
files.download('stock_prediction_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import requests

# Your Finnhub API key
api_key = 'cua9athr01qkpes4gsh0cua9athr01qkpes4gshg'

# Base URL for Finnhub API
base_url = 'https://finnhub.io/api/v1/'

# Example endpoint: Getting the current price for a stock
symbol = 'TTM'  # Example: Apple stock

# Construct the URL for the request
url = f"{base_url}quote?symbol={symbol}&token={api_key}"

# Make the GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    print(f"Current Price Data for {symbol}:")
    print(data)
else:
    print(f"Error: {response.status_code}, Message: {response.text}")


Current Price Data for TTM:
{'c': 0, 'd': None, 'dp': None, 'h': 0, 'l': 0, 'o': 0, 'pc': 0, 't': 0}


In [18]:
import requests
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Your Finnhub API key
api_key = 'cua9athr01qkpes4gsh0cua9athr01qkpes4gshg'

# Base URL for Finnhub API
base_url = 'https://finnhub.io/api/v1/'

# Example endpoint: Getting the current price for a stock
symbol = 'HIG'  # Example: Hartford stock

# Construct the URL for the request
url = f"{base_url}quote?symbol={symbol}&token={api_key}"

# Make the GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()  # Parse the JSON response
    print(f"Current Price Data for {symbol}:")
    print(data)
else:
    print(f"Error: {response.status_code}, Message: {response.text}")

# Extract relevant data from the Finnhub API response
current_price = data['c']  # Current price (c)
open_price = data['o']  # Open price (o)
high_price = data['h']  # High price (h)
low_price = data['l']  # Low price (l)
prev_close = data['pc']  # Previous close (pc)

# For the moving averages, we need the most recent historical close prices
# Assuming you have this data already or can retrieve it. Here we'll use some placeholders.
# You would need the most recent 'Close' prices from your data.
last_close_prices = np.array([110.90, 111.00, 110.80, 110.70, 110.75, 110.85, 111.10, 111.25, 111.30, 111.50])  # Example close prices

# Calculate moving averages
MA_5 = np.mean(last_close_prices[-5:])  # Simple moving average for the last 5 closes
MA_20 = np.mean(last_close_prices[-20:]) if len(last_close_prices) >= 20 else np.mean(last_close_prices)  # MA_20 (fallback to MA_5 if not enough data)

# Create a DataFrame for the current data
current_data = pd.DataFrame({
    'Open': [open_price],
    'High': [high_price],
    'Low': [low_price],
    'Volume': [1000000],  # Placeholder for Volume (replace with actual if available)
    'MA_5': [MA_5],
    'MA_20': [MA_20]
})

# Normalize the data using the same scaler that was used during training
# Ensure you have the trained scaler (for example, save it and reload when needed)
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the current data
scaled_data = scaler.fit_transform(current_data)

# Prepare the input for the LSTM model
look_back = 60  # The model was trained with a look-back of 60 time steps

# Since we only have one data point, we need to repeat it to create a sequence of `look_back` length
X_input = np.repeat(scaled_data, look_back, axis=0).reshape(1, look_back, len(current_data.columns))

# Predict using the trained LSTM model
y_pred = model.predict(X_input)

# Invert scaling for the prediction (predicted close price)
# Reshape y_pred to be 1D, and concatenate it with the last features of X_input
y_pred_rescaled = scaler.inverse_transform(np.concatenate(
    (X_input[0, -1, :-1], y_pred.reshape(-1)), axis=0).reshape(1, -1))[:, -1]
predicted_close = y_pred_rescaled[0]

# Show the predicted close price
print(f"Predicted Close Price: {predicted_close}")


# Step 3: Decision on investment
if predicted_close > current_price:
    print("It might be a good time to invest, as the predicted price is higher than the current price.")
else:
    print("It might not be a good time to invest, as the predicted price is lower than or close to the current price.")


Current Price Data for HIG:
{'c': 111.49, 'd': 0.6, 'dp': 0.5411, 'h': 111.66, 'l': 110.55, 'o': 111.02, 'pc': 110.89, 't': 1737752400}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Close Price: 111.05086436808108
It might not be a good time to invest, as the predicted price is lower than or close to the current price.
